# Faster Text Generation with TensorFlow and XLA

This notebook is a companion to the 🤗 [blog post with the same title](https://huggingface.co/blog/tf-xla-generate). 
It is meant to illustrate how to use XLA with TensorFlow text generation.

It contains two stand-alone examples, one for encoder-decoder models and another for decoder-only models.

⚠️ If you are running this on colab, you might not have access to a GPU. The benefits of XLA are best observed with a GPU!

In [ ]:
# Preparing the environment
!pip install transformers>=4.21.0

In [ ]:
# Stand-alone TF XLA generate example for Encoder-Decoder Models.

# Note: execution times are deeply dependent on hardware.
# If you have a machine with a powerful GPU, I highly recommend you to try this example there!
import time
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# 1. Load model and tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

# 2. Prepare tokenization and generation arguments -- don't forget padding to avoid retracing!
tokenization_kwargs = {"pad_to_multiple_of": 32, "padding": True, "return_tensors": "tf"}
generation_kwargs = {"num_beams": 4, "max_new_tokens": 32}

# 3. Create your XLA generate function a̶n̶d̶ ̶m̶a̶k̶e̶ ̶P̶y̶T̶o̶r̶c̶h̶ ̶e̶a̶t̶ ̶d̶u̶s̶t̶
# This is the only change with respect to original generate workflow!
xla_generate = tf.function(model.generate, jit_compile=True)

# 4. Generate! Remember -- the first call will be slow, but all subsequent calls will be fast if you've done things right.
input_prompts = [
    f"translate English to {language}: I have four cats and three dogs." for language in ["German", "French", "Romanian"]
]
for input_prompt in input_prompts:
    tokenized_inputs = tokenizer([input_prompt], **tokenization_kwargs)
    start = time.time_ns()
    generated_text = xla_generate(**tokenized_inputs, **generation_kwargs)
    end = time.time_ns()
    decoded_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
    print(f"Original prompt -- {input_prompt}")
    print(f"Generated -- {decoded_text}")
    print(f"Execution time -- {(end - start) / 1e6:.1f} ms\n")

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00


In [2]:
!pip install --quiet bitsandbytes
!pip install --quiet accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 12.5 MB/s eta 0:00:00


In [60]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

#######to split  a very large model between cpu and gpu

In [73]:
device_map = {
    "transformer.word_embeddings": 'cpu',
    "transformer.word_embeddings_layernorm": 'cpu',
    "lm_head": "cpu",
    "transformer.h": 'cpu',
    "transformer.ln_f": 'cpu',
}

In [3]:
# Stand-alone TF XLA generate example for Decoder-Only Models.

# Note: execution times are deeply dependent on hardware.
# If you have a machine with a powerful GPU, I highly recommend you to try this example there!
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1. Load model and tokenizer
name = "gpt2"

# remember: decoder-only models need left-padding
model_8bit = AutoModelForCausalLM.from_pretrained(name,load_in_8bit=True,device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(name)
tokenizer.pad_token = tokenizer.eos_token


# 3. Create your XLA generate function a̶n̶d̶ ̶m̶a̶k̶e̶ ̶P̶y̶T̶o̶r̶c̶h̶ ̶e̶a̶t̶ ̶d̶u̶s̶t̶
# This is the only change with respect to original generate workflow!


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), Po

In [6]:
import tensorflow as tf

In [18]:

# 2. Prepare tokenization and generation arguments -- don't forget padding to avoid retracing!
tokenization_kwargs = {"padding": True, "return_tensors": "tf"}
generation_kwargs = {"num_beams": 4, "max_new_tokens": 32}

In [25]:
def most_likely_next_token(input_ids,attention_mask):
    model_output = model_8bit.generate(input_ids=input_ids,attention_mask=attention_mask,**generation_kwargs)
    return tf.argmax(model_output.logits[:, -1, :], axis=-1)

# print("Calling regular function with TensorFlow code...")
# most_likely_next_token(inputs)

In [26]:
# 4. Generate! Remember -- the first call will be slow, but all subsequent calls will be fast if you've done things right.
xla_generate = tf.function(most_likely_next_token, jit_compile=True)
input_prompts = ["The best thing about Spain"]
inputs = tokenizer(input_prompts, **tokenization_kwargs)

In [28]:
inputs = tokenizer(["TensorFlow is"], return_tensors="tf")

In [27]:
start = time.time_ns()
generated_text = xla_generate(inputs['input_ids'],inputs['attention_mask'])
end = time.time_ns()
decoded_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/tensorflow/python/util/traceback_utils.py:153 in          │
│ error_handler                                                                                    │
│                                                                                                  │
│   150 │     return fn(*args, **kwargs)                                                           │
│   151 │   except Exception as e:                                                                 │
│   152 │     filtered_tb = _process_traceback_frames(e.__traceback__)                             │
│ ❱ 153 │     raise e.with_traceback(filtered_tb) from None                                        │
│   154 │   finally:                                                                               │
│   155 │     del filtered_tb                                                                      │
│   156                                                                                            │
│                                                                                                  │
│ /tmp/__autograph_generated_file_l36g6h7.py:10 in tf__most_likely_next_token                      │
│                                                                                                  │
│    7 │   │   │   with ag__.FunctionScope('most_likely_next_token', 'fscope', ag__.ConversionO    │
│    8 │   │   │   │   do_return = False                                                           │
│    9 │   │   │   │   retval_ = ag__.UndefinedReturnValue()                                       │
│ ❱ 10 │   │   │   │   model_output = ag__.converted_call(ag__.ld(model_8bit).generate, (), dic    │
│   11 │   │   │   │   try:                                                                        │
│   12 │   │   │   │   │   do_return = True                                                        │
│   13 │   │   │   │   │   retval_ = ag__.converted_call(ag__.ld(tf).argmax, (ag__.ld(model_out    │
│                                                                                                  │
│ /tmp/__autograph_generated_fileqp63r20e.py:15 in tf__decorate_context                            │
│                                                                                                  │
│   12 │   │   │   │   with ag__.ld(ctx_factory)():                                                │
│   13 │   │   │   │   │   try:                                                                    │
│   14 │   │   │   │   │   │   do_return = True                                                    │
│ ❱ 15 │   │   │   │   │   │   retval_ = ag__.converted_call(ag__.ld(func), tuple(ag__.ld(args)    │
│   16 │   │   │   │   │   except:                                                                 │
│   17 │   │   │   │   │   │   do_return = False                                                   │
│   18 │   │   │   │   │   │   raise                                                               │
│                                                                                                  │
│ /tmp/__autograph_generated_filew3bn0fkg.py:221 in tf__generate                                   │
│                                                                                                  │
│   218 │   │   │   │                                                                              │
│   219 │   │   │   │   def else_body_8():                                                         │
│   220 │   │   │   │   │   pass                                                                   │
│ ❱ 221 │   │   │   │   ag__.if_stmt(ag__.not_(ag__.ld(self).

In [30]:
print(f"Original prompt -- {input_prompt}")
print(f"Generated -- {decoded_text}")
print(f"Execution time -- {(end - start) / 1e6:.1f} ms\n")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/tensorflow/python/util/traceback_utils.py:153 in          │
│ error_handler                                                                                    │
│                                                                                                  │
│   150 │     return fn(*args, **kwargs)                                                           │
│   151 │   except Exception as e:                                                                 │
│   152 │     filtered_tb = _process_traceback_frames(e.__traceback__)                             │
│ ❱ 153 │     raise e.with_traceback(filtered_tb) from None                                        │
│   154 │   finally:                                                                               │
│   155 │     del filtered_tb                                                                      │
│   156                                                                                            │
│                                                                                                  │
│ /tmp/__autograph_generated_fileddeo6_q0.py:10 in tf__mt                                          │
│                                                                                                  │
│    7 │   │   │   with ag__.FunctionScope('mt', 'fscope', ag__.ConversionOptions(recursive=Tru    │
│    8 │   │   │   │   do_return = False                                                           │
│    9 │   │   │   │   retval_ = ag__.UndefinedReturnValue()                                       │
│ ❱ 10 │   │   │   │   model_output = ag__.converted_call(ag__.ld(model), (ag__.ld(inputs),), N    │
│   11 │   │   │   │   try:                                                                        │
│   12 │   │   │   │   │   do_return = True                                                        │
│   13 │   │   │   │   │   retval_ = ag__.converted_call(ag__.ld(tf).argmax, (ag__.ld(model_out    │
│                                                                                                  │
│ /tmp/__autograph_generated_filejs376p4e.py:53 in tf____call__                                    │
│                                                                                                  │
│   50 │   │   │   │   retval_ = ag__.UndefinedReturnValue()                                       │
│   51 │   │   │   │   try:                                                                        │
│   52 │   │   │   │   │   do_return = True                                                        │
│ ❱ 53 │   │   │   │   │   retval_ = ag__.converted_call(ag__.converted_call(ag__.ld(super), ()    │
│   54 │   │   │   │   except:                                                                     │
│   55 │   │   │   │   │   do_return = False                                                       │
│   56 │   │   │   │   │   raise                                                                   │
│                                                                                                  │
│ /tmp/__autograph_generated_filebecm91ef.py:219 in tf____call__                                   │
│                                                                                                  │
│   216 │   │   │   │   │   ag__.if_stmt(ag__.ld(can_use_iterator), if_body_9, else_body_9, get_   │
│   217 │   │   │   │   outputs = ag__.Undefined('outputs')                                        │
│   218 │   │   │   │   final_iterator = ag__.Undefined('final_iterator')                          │
│ ❱ 219 │   │   │   │   ag__.if_stmt(ag__.ld(is_list), if_bod

In [ ]:
decoded_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
print(f"Original prompt -- {input_prompt}")
print(f"Generated -- {decoded_text}")
print(f"Execution time -- {(end - start) / 1e6:.1f} ms\n")